## A wrapper for OPA within AQUA

In [2]:
from aqua import OPAgenerator

Initialize the OPA generator class, feeding it with the most of the information you need. Which variable, which model source and where to store the data.  Please change the outdir if you want to try this notebook!

In [3]:
model = 'IFS'
exp = 'tco2559-ng5'
source = 'ICMGG_atm2d'
outdir = '/home/b/b382321/scratch/one_pass'
tmpdir = '/home/b/b382321/scratch/one_pass'

In [4]:
opaopa = OPAgenerator(model=model, exp=exp, source=source, var='2t', frequency='daily',
            outdir=outdir, tmpdir=tmpdir, definitive=False, loglevel ='INFO')


2023-05-17 17:38:04 :: opa_generator :: WARNING  -> IMPORTANT: no file will be created, this is a dry run
2023-05-17 17:38:04 :: opa_generator :: WARNING  -> Variable(s) to be processed: 2t
2023-05-17 17:38:04 :: create_folder :: WARNING  -> Creating folder /home/b/b382321/scratch/one_pass/IFS/tco2559-ng5/daily
2023-05-17 17:38:04 :: create_folder :: WARNING  -> Folder /home/b/b382321/scratch/one_pass already exists


Then you can retrieve the data using the `Reader`. This is based on a streaming generator so far, since we are using data from NextGEMS. It will be later replaced with direct access from `FDB`. It also gives a look at the data so that it can infer the timedelta

In [5]:
opaopa.retrieve()

2023-05-17 17:38:10 :: opa_generator :: INFO     -> Accessing catalog for IFS-tco2559-ng5-ICMGG_atm2d...
2023-05-17 17:38:29 :: opa_generator :: INFO     -> Getting the timedelta to inform the OPA...
2023-05-17 17:38:35 :: Reader :: INFO     -> tprate: m --> m s**-1
2023-05-17 17:38:35 :: Reader :: INFO     -> tprate: corrected dividing by accumulation time 3600 s
2023-05-17 17:38:35 :: Reader :: INFO     -> Fixing tp to tprate. Unit fix: factor=0.000278, offset=0.000000
2023-05-17 17:38:35 :: Reader :: INFO     -> Grib attributes for cprate: {'paramId': '172143', 'long_name': 'Mean convective precipitation rate', 'units': 'm s**-1', 'cfVarName': 'cprate', 'shortName': 'cprate'}
2023-05-17 17:38:35 :: Reader :: INFO     -> var172143: m --> m s**-1
2023-05-17 17:38:35 :: Reader :: INFO     -> var172143: corrected dividing by accumulation time 3600 s
2023-05-17 17:38:35 :: Reader :: INFO     -> Fixing cp to var172143. Unit fix: factor=0.000278, offset=0.000000
2023-05-17 17:38:35 :: Read

This will launch the OPA computation, if the `definitive` flag is `true` it will take ages and probably break the notebook. Here, we only print the sequential access to the data from the `Reader` (note that to print the sequential access the loglevel should be set to `INFO` or `DEBUG`).

In [ ]:
opaopa.generate_opa()

You can then add an entry to the catalog so that you can access these data from somewhere else.

In [7]:
opaopa.create_catalog_entry()

2023-05-17 17:39:26 :: opa_generator :: WARNING  -> Creating catalog entry IFS tco2559-ng5 tmp-opa-daily


In [8]:
from aqua import Reader
reader = Reader(model="IFS", exp="tco2559-ng5", source="tmp-opa-daily",areas=False)

Remember that in order to really access and work with data, a run with `definitive` set to `True` is needed.

The following command removes the entries (and in the future also the files), once all the computation have been safely concluded

In [9]:
opaopa.clean()

2023-05-17 17:39:32 :: opa_generator :: WARNING  -> Removing catalog entry IFS tco2559-ng5 tmp-opa-daily
